In [99]:
import json
from pprint import pprint
from pymongo import MongoClient
import json
import copy
import re
import os
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation
from hazm import Normalizer, word_tokenize, stopwords_list, Stemmer
import copy
import collections
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import Union

import arabic_reshaper
import demoji
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from bidi.algorithm import get_display
from hazm import Normalizer, sent_tokenize, word_tokenize
from loguru import logger
from PIL import Image
from tqdm import tqdm
from wordcloud import WordCloud
from loguru import logger

In [2]:
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "pytopia"
COLLECTION_NAME = "messages"
START_DATE = '2022-07-01T00:00:00'
END_DATE = '2023-07-31T23:59:59'

In [27]:
client = MongoClient()
db = client[DB_NAME]
cursor = db[COLLECTION_NAME].find({})
# for document in cursor:
#     pprint(document)
#     break

In [4]:
chat_data = [doc for doc in cursor]

In [100]:


normalizer = Normalizer()

# load stopwords
logger.info(f"Loading stopwords from {'stopwords.txt'}")
stop_words = open('stopwords.txt').readlines()
stop_words = map(str.strip, stop_words)
stop_words = set(map(normalizer.normalize, stop_words))

def remove_stopwords(text):
    tokens = word_tokenize(normalizer.normalize(text))
    tokens = list(filter(lambda item: item not in stop_words, tokens))
    return ' '.join(tokens)

def de_emojify(text):
    regrex_pattern = re.compile(pattern="[\u2069\u2066]+", flags=re.UNICODE)
    text = regrex_pattern.sub('', text)
    return demoji.replace(text, " ")

def preprocess_text(msg):
    text_content = ''
    if isinstance(msg, list):
        for sub_msg in msg:
            if isinstance(sub_msg, str):
                text_content += f" {remove_stopwords(sub_msg)}"
            elif isinstance(sub_msg, dict) and sub_msg['type'] in {
                'text_link', 'bold', 'italic',
                'hashtag', 'mention', 'pre'
            }:
                text_ = remove_stopwords(sub_msg['text'])
                text_content += f" {text_}"
    else:
        text_content += f" {remove_stopwords(msg)}"

    tokens = list(word_tokenize(normalizer.normalize(text_content)))
    return tokens

2024-06-10 16:05:24.941 | INFO     | __main__:<module>:4 - Loading stopwords from stopwords.txt


In [49]:
def preprocess(input_docs):
    docs = copy.deepcopy(input_docs)
    for doc in docs:
        if not doc.get('text'):
            continue
        content = doc['text']
        doc['text'] = preprocess_text(content)
    return docs

In [50]:
data[180]["text"]

['🛑بچه ها یه نکته که البته خیلی هم مهم نیست \nشما وقتی به صورت manually   فایل رو از این لینک دانلود می کنید\n',
 {'type': 'link',
  'text': 'https://docs.microsoft.com/en-us/windows/wsl/install-manual'},
 '\nطبیعتا باید با یک فایل تک با پسوند appx رو به رو باشید\nحالا بعضی بچه ها که با IDM دانلود می کنن IDM پسوند فایل رو به ZIP تغییر میده  و بچه ها هم اون فایل رو باز می کنن و سردرگم  میشن\nکه مساله خاصی نیست خودتون دوباره پسوند فایل رو دستی از zip بکنید appx  و wsl اتون که الان یه تک فایل هست رو نصب کنید']

In [51]:
p_data = preprocess(data)

In [54]:
p_data[400]

{'_id': ObjectId('6661636c9aff4f6795898336'),
 'id': 1041,
 'type': 'message',
 'date': '2021-07-06T12:35:21',
 'date_unixtime': '1625562321',
 'from': 'MEHDI',
 'from_id': 'user310785297',
 'reply_to_message_id': 629,
 'text': ['اره',
  'میدونم',
  'یهو',
  'انقلابی',
  'تصمیم',
  'گرفتم',
  'سوییچ',
  'ابونتو',
  '😂'],
 'text_entities': [{'type': 'plain',
   'text': 'اره میدونم میشه ولی من یهو خیلی انقلابی تصمیم گرفتم سوییچ کنم به ابونتو 😂'}]}

In [55]:
def create_index_dict(docs):
    
    index = {}
    for doc in docs:    
        for pos, token in enumerate(doc['text']):
            if token not in index:
                index[token] = {
                'total_freq': 1,
                 'docs': {
                       doc['id']: {
                           'count_of_word': 1,
                           'positions': [pos]
                           }
                    }
                }
            else:
                index[token]['total_freq'] += 1  
                if doc['id'] not in index[token]['docs']:
                    index[token]['docs'][doc['id']] = {}
                    index[token]['docs'][doc['id']]['count_of_word'] = 1
                    index[token]['docs'][doc['id']]['positions'] = [pos]
                else:
                    index[token]['docs'][doc['id']]['count_of_word'] += 1 
                    index[token]['docs'][doc['id']]['positions'].append(pos)   
    
    # create champion lists             
    for word in index:
        champ_list = sorted(index[word]['docs'], key=lambda x: index[word]['docs'][x]['count_of_word'], reverse=True)
        index[word]['champions'] = champ_list[:len(champ_list) // 5]
    return index

In [59]:
pre_processed_docs = p_data

In [56]:
index = create_index_dict(p_data)

In [83]:
def simple_process(docs, tokens):
    docs_score = {}
    for token in tokens:
        if token in index:
            for doc_id in index[token]['docs']:
                    if doc_id not in docs_score.keys():
                        docs_score[doc_id] = 1
                    else:
                        docs_score[doc_id] += 1
   
    docs_score = sorted(docs_score.items(), key=lambda doc_score: doc_score[1], reverse=True)
    result = [x for x,_ in docs_score]
    return result


def exceptions_process(docs, tokens):
    removal_docs = []
    for token in tokens:
        rmv_docIds = index[token]['docs'].keys()
        for doc_id in rmv_docIds:  
             if removal_docs.count(doc_id) == 0:
                    removal_docs.append(doc_id)

    result = []            
    for doc_id in docs:
        if removal_docs.count(doc_id) == 0:
            result.append(doc_id)

    return result

def doc_contain_phrase(docId, phrase):
   # we need just a sequence of numbers in positions to find a doc contain special phrase",
    tokens_lst = phrase.split()
    positions = []
    for token in tokens_lst:
        positions.append(list(index[token]['docs'][docId]['positions']))

    for i in range (len(positions[0])):
        flag = True
        index_of_first_token = positions[0][i]
        for j in range (len(positions)):
            if positions[j].count(index_of_first_token + j) == 0:
                flag = False
        if flag :
            return True
    return flag

def phrasal_process(docs, phrases):
    posting_lists = []
    result = []
    for phrase in phrases:
        words = phrase.split()
        for word in words:
            if word in index:
                posting_lists.append(list(index[word]['docs'].keys()))
    
        intersection_doc_ids = set.intersection(*map(set, posting_lists))

        for docId in intersection_doc_ids:
            if doc_contain_phrase(docId, phrase):     
                result.append(docId)
    return result


def process_query(query):
    
    exception_tokens = re.findall(r'\!\s(\w+)', query)
    phrasal_tokens = re.findall(r'"([^"]*)"', query)
    raw_tokens = re.sub(r'\!\s\w+', '', query)
    raw_tokens = re.sub(r'"[^"]*"', '', raw_tokens)
    
    simple_tokens = preprocess_text(raw_tokens)
    print(exception_tokens, phrasal_tokens, raw_tokens)
    result = None
    # sorted documents by relevance after processing simple tokens\n",
    if len(simple_tokens) > 0:
        result = simple_process(pre_processed_docs, simple_tokens)
        
    # removing the items that included exception tokens, kind of post filter
    if len(exception_tokens) > 0:
        removed_result = exceptions_process(result, exception_tokens)
        if len(removed_result) > 0 and len(result) > 0:
            result = [x for x in result if x in removed_result]  
            
    # Intersect of results with those containing phrasal queries\n",
    if len(phrasal_tokens) > 0:
        phrasal_result = phrasal_process(pre_processed_docs, phrasal_tokens)
        
        if result is None or len(result) == 0:
            return phrasal_result
        result = [x for x in result if x in phrasal_result]  

    return result

def query_print(query, content=False, max_cnt=20):
    results = process_query(query)[:max_cnt]
    if len(results) == 0:
        print("نتیجه ای یافت نشد")
    for rank, doc in enumerate(results):
        if doc == None:
            continue
        print(50*'=')
        print(f'Rank: {rank + 1}, docID: {doc}')
        for dict_ in data:
            if dict_["id"] == doc:
            
                print(f'From: {dict_["from"]}')
                print(f'Date: {dict_["date"]}')
                
                print(f'{dict_["text"]}')
                break

In [85]:
query_print("مهلت ثبت نام")

[] [] مهلت ثبت نام
Rank: 1, docID: 4648
From: Mahzad Sharif
Date: 2021-07-13T11:14:39
پس این مهلت ثبت نام تا ۲۳ تیر فقط برای انلاین ها هست؟هنوز برای افرادی که میخوان افلاین شرکت کنن اعلام نشده نحوه ثبت نام؟
همش میترسم جا بمون واقعا😅😅🙏🙏
Rank: 2, docID: 5613
From: Mahzad Sharif
Date: 2021-07-15T23:05:34
تعداد که بله  محدودیت نداره احتمالا منظورم مهلت ثبت نام بود عزیزم🙂🌷 مهلت انلاین تو پیام پین تا انتهای ۲۳ تیر بود تا جایی که یادمه. اون که تموم شده ددلاینش فک کنم!!!
Rank: 3, docID: 6226
From: Masoud
Date: 2021-07-16T18:21:59
دوستان مهلت ثبت نام آفلاین باز می مونه یا تایم محدود داره اونم؟
Rank: 4, docID: 14133
From: Huny
Date: 2021-12-20T09:47:23
['💎 تنها سه روز تا برگزاری دومین دوره ', {'type': 'bold', 'text': 'رویداد بین\u200cالمللی ReACT'}, ' باقی مانده است.\n\n💫 این رویداد با محوریت بررسی فناوری\u200cهای نوین مهندسی برق، امسال هم افتخار میزبانی از فعالان علمی و صنعتی شناخته\u200cشده از دانشگاه\u200cها و شرکت\u200cهای نام\u200cآشنای سراسر دنیا را دارد.\n\n⭐️ رویداد امسال به مدت چهار روز

# Production Code: 

In [101]:


class TextPreprocessor:
    def __init__(self, stopwords_file):
        self.normalizer = Normalizer()
        self.stop_words = self.load_stopwords(stopwords_file)
        
    def load_stopwords(self, file_path):
        logger.info(f"Loading stopwords from {file_path}")
        with open(file_path) as f:
            return {self.normalizer.normalize(line.strip()) for line in f}
        
    def remove_stopwords(self, text):
        tokens = word_tokenize(self.normalizer.normalize(text))
        return ' '.join(token for token in tokens if token not in self.stop_words)
    
    def de_emojify(self, text):
        # Remove certain Unicode characters and replace emojis
        pattern = re.compile(r"[\u2069\u2066]+", re.UNICODE)
        text = pattern.sub('', text)
        return demoji.replace(text, " ")
    
    def preprocess_text(self, message):
        text_content = ''
        if isinstance(message, list):
            for sub_message in message:
                if isinstance(sub_message, str):
                    text_content += f" {self.remove_stopwords(sub_message)}"
                elif isinstance(sub_message, dict) and sub_message.get('type') in {
                    'text_link', 'bold', 'italic', 'hashtag', 'mention', 'pre'
                }:
                    text_content += f" {self.remove_stopwords(sub_message['text'])}"
        else:
            text_content += f" {self.remove_stopwords(message)}"
        
        tokens = word_tokenize(self.normalizer.normalize(text_content))
        return tokens
    
    def preprocess(self, input_docs):
        processed_docs = copy.deepcopy(input_docs)
        for doc in processed_docs:
            if 'text' in doc and doc['text']:
                doc['text'] = self.preprocess_text(doc['text'])
        return processed_docs


In [113]:
data[0]

{'_id': ObjectId('6661636c9aff4f67958981a6'),
 'id': 2,
 'type': 'message',
 'date': '2021-07-05T23:22:36',
 'date_unixtime': '1625514756',
 'from': 'a.20',
 'from_id': 'user1278735911',
 'text': ['آقای حجازی \nاگر احیانا تعداد نفرات متقاضی شرکت در دوره بیش از ظرفیت پلتفرم های برگزاری دوره بود، خواهشاً دوره را به صورت آفلاین برگزار کنید\n\nما نمی\u200cخواهیم دوره رو  از دست بدیم \n',
  {'type': 'mention', 'text': '@Aliii_H93'},
  ''],
 'text_entities': [{'type': 'plain',
   'text': 'آقای حجازی \nاگر احیانا تعداد نفرات متقاضی شرکت در دوره بیش از ظرفیت پلتفرم های برگزاری دوره بود، خواهشاً دوره را به صورت آفلاین برگزار کنید\n\nما نمی\u200cخواهیم دوره رو  از دست بدیم \n'},
  {'type': 'mention', 'text': '@Aliii_H93'},
  {'type': 'plain', 'text': ''}]}

In [102]:
# Example usage:
preprocessor = TextPreprocessor('stopwords.txt')
p_data = preprocessor.preprocess(data)

2024-06-10 16:05:43.712 | INFO     | __main__:load_stopwords:7 - Loading stopwords from stopwords.txt


In [119]:
data[4:6]

[{'_id': ObjectId('6661636c9aff4f67958981aa'),
  'id': 6,
  'type': 'message',
  'date': '2021-07-05T23:26:28',
  'date_unixtime': '1625514988',
  'from': 'Sima',
  'from_id': 'user622339736',
  'reply_to_message_id': 2,
  'text': 'موافقم\nمن حتی نتونستم داخل کلاس جوین بشم آفلاین هم برام باز نشد',
  'text_entities': [{'type': 'plain',
    'text': 'موافقم\nمن حتی نتونستم داخل کلاس جوین بشم آفلاین هم برام باز نشد'}]},
 {'_id': ObjectId('6661636c9aff4f67958981ab'),
  'id': 9,
  'type': 'message',
  'date': '2021-07-05T23:26:37',
  'date_unixtime': '1625514997',
  'from': 'Sobhan Razyani',
  'from_id': 'user72383515',
  'reply_to_message_id': 4,
  'text': 'ممنونم. Jady؟',
  'text_entities': [{'type': 'plain', 'text': 'ممنونم. Jady؟'}]}]

In [111]:
# import re

# class DocumentProcessor:
#     def __init__(self, index, data, pre_processed_docs):
#         self.index = index
#         self.data = data
#         self.pre_processed_docs = pre_processed_docs

#     @staticmethod
#     def create_index_dict(docs):
#         index = {}
#         for doc in docs:
#             for pos, token in enumerate(doc['text'].split()):  # Split text into tokens
#                 if token not in index:
#                     index[token] = {
#                         'total_freq': 1,
#                         'docs': {
#                             doc['id']: {
#                                 'count_of_word': 1,
#                                 'positions': [pos]
#                             }
#                         }
#                     }
#                 else:
#                     index[token]['total_freq'] += 1
#                     if doc['id'] not in index[token]['docs']:
#                         index[token]['docs'][doc['id']] = {
#                             'count_of_word': 1,
#                             'positions': [pos]
#                         }
#                     else:
#                         index[token]['docs'][doc['id']]['count_of_word'] += 1
#                         index[token]['docs'][doc['id']]['positions'].append(pos)

#         # Create champion lists
#         for word in index:
#             champ_list = sorted(index[word]['docs'], key=lambda x: index[word]['docs'][x]['count_of_word'], reverse=True)
#             index[word]['champions'] = champ_list[:len(champ_list) // 5]
#         return index

#     def simple_process(self, tokens):
#         docs_score = {}
#         for token in tokens:
#             if token in self.index:
#                 for doc_id in self.index[token]['docs']:
#                     if doc_id not in docs_score:
#                         docs_score[doc_id] = 1
#                     else:
#                         docs_score[doc_id] += 1

#         docs_score = sorted(docs_score.items(), key=lambda doc_score: doc_score[1], reverse=True)
#         result = [x for x, _ in docs_score]
#         return result

#     def exceptions_process(self, docs, tokens):
#         removal_docs = []
#         for token in tokens:
#             rmv_doc_ids = self.index[token]['docs'].keys()
#             for doc_id in rmv_doc_ids:
#                 if doc_id not in removal_docs:
#                     removal_docs.append(doc_id)

#         result = [doc_id for doc_id in docs if doc_id not in removal_docs]
#         return result

#     def doc_contain_phrase(self, doc_id, phrase):
#         tokens_lst = phrase.split()
#         positions = []
#         for token in tokens_lst:
#             positions.append(list(self.index[token]['docs'][doc_id]['positions']))

#         for i in range(len(positions[0])):
#             flag = True
#             index_of_first_token = positions[0][i]
#             for j in range(len(positions)):
#                 if index_of_first_token + j not in positions[j]:
#                     flag = False
#             if flag:
#                 return True
#         return False

#     def phrasal_process(self, phrases):
#         posting_lists = []
#         result = []
#         for phrase in phrases:
#             words = phrase.split()
#             for word in words:
#                 if word in self.index:
#                     posting_lists.append(list(self.index[word]['docs'].keys()))

#             intersection_doc_ids = set.intersection(*map(set, posting_lists))

#             for doc_id in intersection_doc_ids:
#                 if self.doc_contain_phrase(doc_id, phrase):
#                     result.append(doc_id)
#         return result

#     def process_query(self, query):
#         exception_tokens = re.findall(r'\!\s(\w+)', query)
#         phrasal_tokens = re.findall(r'"([^"]*)"', query)
#         raw_tokens = re.sub(r'\!\s\w+', '', query)
#         raw_tokens = re.sub(r'"[^"]*"', '', raw_tokens)
#         simple_tokens = self.preprocess_text(raw_tokens)

#         result = None
#         if simple_tokens:
#             result = self.simple_process(simple_tokens)

#         if exception_tokens:
#             removed_result = self.exceptions_process(result, exception_tokens)
#             if removed_result and result:
#                 result = [x for x in result if x in removed_result]

#         if phrasal_tokens:
#             phrasal_result = self.phrasal_process(phrasal_tokens)
#             if result is None or not result:
#                 return phrasal_result
#             result = [x for x in result if x in phrasal_result]

#         return result

#     def query_print(self, query, content=False, max_cnt=20):
#         results = self.process_query(query)[:max_cnt]
#         if not results:
#             print("نتیجه ای یافت نشد")
#         for rank, doc in enumerate(results):
#             if doc is None:
#                 continue
#             print(50 * '=')
#             print(f'Rank: {rank + 1}, docID: {doc}')
#             for dict_ in self.data:
#                 if dict_["id"] == doc:
#                     print(f'From: {dict_["from"]}')
#                     print(f'Date: {dict_["date"]}')
#                     print(f'{dict_["text"]}')
#                     break

#     def preprocess_text(self, text):
#         # Implement your text preprocessing here
#         return text.split()


In [112]:
# # Usage example
# index = {
#     'example': {
#         'docs': {
#             1: {'positions': [1, 2]},
#             2: {'positions': [3]}
#         }
#     }
# }
# data = [
#     {"id": 1, "from": "User1", "date": "2023-01-01", "text": "Example text 1"},
#     {"id": 2, "from": "User2", "date": "2023-01-02", "text": "Example text 2"}
# ]
# pre_processed_docs = [1, 2]

# processor = DocumentProcessor(index, data, pre_processed_docs)
# processor.query_print('example query')